In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
def to_rgb(img):
    if img.ndim == 2:  # (H, W)
        img = np.expand_dims(img, axis=-1)  # (H, W, 1)
    if img.shape[-1] == 1:
        img = np.repeat(img, 3, axis=-1)  # (H, W, 3)
    return img

In [4]:
# 4. Data generators
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=5,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest',
    preprocessing_function=to_rgb,
    validation_split=0.3
)


test_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=to_rgb
)

In [5]:
train_data = train_gen.flow_from_directory(
    'train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_data = train_gen.flow_from_directory(
    'train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

test_data = test_datagen.flow_from_directory(
    'test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


Found 20099 images belonging to 7 classes.
Found 8610 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [6]:
# 5. Early stopping callback
from tensorflow.keras.callbacks import EarlyStopping
callback = EarlyStopping(
    monitor='val_loss',
    min_delta=0.0001,
    patience=20,
    verbose=1,
    mode="auto",
    restore_best_weights=False
)

In [7]:
# 6. Compute class weights
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_data.classes),
    y=train_data.classes
)
class_weights_dict = dict(enumerate(class_weights))

In [10]:
# 7. Build MobileNetV2 model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load base model
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze base model

# Add classifier head
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
x = Dropout(0.3)(x)
x = Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
x = Dropout(0.3)(x)
output = Dense(train_data.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# Compile model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# 8. Train the head
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=15,
    callbacks=[callback],
    class_weight=class_weights_dict
)

Epoch 1/15
629/629 ━━━━━━━━━━━━━━━━━━━━ 578s 913ms/step - accuracy: 0.1661 - loss: 2.1091 - val_accuracy: 0.2401 - val_loss: 2.0281
Epoch 2/15
629/629 ━━━━━━━━━━━━━━━━━━━━ 978s 2s/step - accuracy: 0.2183 - loss: 2.0107 - val_accuracy: 0.3134 - val_loss: 1.9294
Epoch 3/15
629/629 ━━━━━━━━━━━━━━━━━━━━ 620s 986ms/step - accuracy: 0.2657 - loss: 1.9408 - val_accuracy: 0.3383 - val_loss: 1.8752
Epoch 4/15
629/629 ━━━━━━━━━━━━━━━━━━━━ 1106s 2s/step - accuracy: 0.2940 - loss: 1.8883 - val_accuracy: 0.3269 - val_loss: 1.8471
Epoch 5/15
629/629 ━━━━━━━━━━━━━━━━━━━━ 1297s 2s/step - accuracy: 0.3030 - loss: 1.8542 - val_accuracy: 0.3574 - val_loss: 1.7850
Epoch 6/15
629/629 ━━━━━━━━━━━━━━━━━━━━ 446s 708ms/step - accuracy: 0.3171 - loss: 1.8211 - val_accuracy: 0.3519 - val_loss: 1.7760
Epoch 7/15
629/629 ━━━━━━━━━━━━━━━━━━━━ 446s 709ms/step - accuracy: 0.3325 - loss: 1.7956 - val_accuracy: 0.3287 - val_loss: 1.8019
Epoch 8/15
629/629 ━━━━━━━━━━━━━━━━━━━━ 442s 702ms/step - accuracy: 0.3427 - loss: 

In [11]:
model.save('Not_model.h5')

In [13]:
# 9. Fine-tuning
fine_tune_at = 100  # Unfreeze last 100 layers
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False
for layer in base_model.layers[fine_tune_at:]:
    layer.trainable = True

# Recompile for fine-tuning
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Train further
fine_tune_history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=30,
    callbacks=[callback],
    class_weight=class_weights_dict
)

model.save('model_15_epocs.h5')

Epoch 1/30
629/629 ━━━━━━━━━━━━━━━━━━━━ 497s 777ms/step - accuracy: 0.3814 - loss: 1.6588 - val_accuracy: 0.4089 - val_loss: 1.6172
Epoch 2/30
629/629 ━━━━━━━━━━━━━━━━━━━━ 488s 775ms/step - accuracy: 0.4074 - loss: 1.5965 - val_accuracy: 0.4235 - val_loss: 1.5668
Epoch 3/30
629/629 ━━━━━━━━━━━━━━━━━━━━ 489s 778ms/step - accuracy: 0.4330 - loss: 1.5430 - val_accuracy: 0.4534 - val_loss: 1.5254
Epoch 4/30
629/629 ━━━━━━━━━━━━━━━━━━━━ 487s 775ms/step - accuracy: 0.4415 - loss: 1.5042 - val_accuracy: 0.4733 - val_loss: 1.4704
Epoch 5/30
629/629 ━━━━━━━━━━━━━━━━━━━━ 487s 775ms/step - accuracy: 0.4624 - loss: 1.4626 - val_accuracy: 0.4728 - val_loss: 1.4587
Epoch 6/30
629/629 ━━━━━━━━━━━━━━━━━━━━ 686s 1s/step - accuracy: 0.4718 - loss: 1.4235 - val_accuracy: 0.4885 - val_loss: 1.4187
Epoch 7/30
629/629 ━━━━━━━━━━━━━━━━━━━━ 535s 850ms/step - accuracy: 0.4819 - loss: 1.3953 - val_accuracy: 0.4997 - val_loss: 1.3898
Epoch 8/30
629/629 ━━━━━━━━━━━━━━━━━━━━ 489s 778ms/step - accuracy: 0.4962 - lo

In [14]:
loss, accuracy = model.evaluate(test_data)
print(f"Test Accuracy: {accuracy}")

225/225 ━━━━━━━━━━━━━━━━━━━━ 129s 575ms/step - accuracy: 0.5712 - loss: 1.2219
Test Accuracy: 0.5711897611618042


In [33]:
# 9. Fine-tuning
fine_tune_at = 100  # Unfreeze last 100 layers
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False
for layer in base_model.layers[fine_tune_at:]:
    layer.trainable = True

# Recompile for fine-tuning
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Train further
fine_tune_history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=5,
    callbacks=[callback],
    class_weight=class_weights_dict
)

model.save('model_35_epocs.h5')

Epoch 1/5
629/629 ━━━━━━━━━━━━━━━━━━━━ 517s 805ms/step - accuracy: 0.6103 - loss: 1.0191 - val_accuracy: 0.5811 - val_loss: 1.2018
Epoch 2/5
629/629 ━━━━━━━━━━━━━━━━━━━━ 497s 791ms/step - accuracy: 0.6119 - loss: 1.0126 - val_accuracy: 0.5875 - val_loss: 1.1814
Epoch 3/5
629/629 ━━━━━━━━━━━━━━━━━━━━ 494s 786ms/step - accuracy: 0.6204 - loss: 1.0000 - val_accuracy: 0.5871 - val_loss: 1.1921
Epoch 4/5
629/629 ━━━━━━━━━━━━━━━━━━━━ 538s 855ms/step - accuracy: 0.6231 - loss: 0.9864 - val_accuracy: 0.5841 - val_loss: 1.2020
Epoch 5/5
629/629 ━━━━━━━━━━━━━━━━━━━━ 607s 964ms/step - accuracy: 0.6249 - loss: 0.9768 - val_accuracy: 0.5909 - val_loss: 1.1808


In [36]:
loss, accuracy = model.evaluate(test_data)
print(f"Test Accuracy: {accuracy}")

225/225 ━━━━━━━━━━━━━━━━━━━━ 80s 357ms/step - accuracy: 0.5848 - loss: 1.1935
Test Accuracy: 0.5848425626754761


In [59]:
# 9. Fine-tuning
fine_tune_at = 100  # Unfreeze last 100 layers
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False
for layer in base_model.layers[fine_tune_at:]:
    layer.trainable = True

# Recompile for fine-tuning
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Train further
fine_tune_history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=7,
    callbacks=[callback],
    class_weight=class_weights_dict
)

model.save('model_42_epocs.h5')

loss, accuracy = model.evaluate(test_data)
print(f"Test Accuracy: {accuracy}")

Epoch 1/7
629/629 ━━━━━━━━━━━━━━━━━━━━ 484s 759ms/step - accuracy: 0.6281 - loss: 0.9748 - val_accuracy: 0.5912 - val_loss: 1.1746
Epoch 2/7
629/629 ━━━━━━━━━━━━━━━━━━━━ 482s 766ms/step - accuracy: 0.6345 - loss: 0.9666 - val_accuracy: 0.5884 - val_loss: 1.1763
Epoch 3/7
629/629 ━━━━━━━━━━━━━━━━━━━━ 479s 762ms/step - accuracy: 0.6373 - loss: 0.9571 - val_accuracy: 0.5884 - val_loss: 1.1987
Epoch 4/7
629/629 ━━━━━━━━━━━━━━━━━━━━ 482s 766ms/step - accuracy: 0.6421 - loss: 0.9456 - val_accuracy: 0.5890 - val_loss: 1.1868
Epoch 5/7
629/629 ━━━━━━━━━━━━━━━━━━━━ 477s 758ms/step - accuracy: 0.6431 - loss: 0.9457 - val_accuracy: 0.5944 - val_loss: 1.1756
Epoch 6/7
629/629 ━━━━━━━━━━━━━━━━━━━━ 477s 758ms/step - accuracy: 0.6512 - loss: 0.9293 - val_accuracy: 0.5999 - val_loss: 1.1766
Epoch 7/7
629/629 ━━━━━━━━━━━━━━━━━━━━ 652s 1s/step - accuracy: 0.6503 - loss: 0.9245 - val_accuracy: 0.5965 - val_loss: 1.1764


225/225 ━━━━━━━━━━━━━━━━━━━━ 86s 381ms/step - accuracy: 0.5904 - loss: 1.1991
Test Accuracy: 0.5904151797294617


In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np

def predict_emotion(image_path, model, class_indices):
    # 1. Load image
    img = image.load_img(image_path, target_size=(224, 224), color_mode='rgb')

    # 2. Convert to array
    img_array = image.img_to_array(img)

    # 3. Expand dimensions to match batch format
    img_array = np.expand_dims(img_array, axis=0)

    # 4. Preprocess input for MobileNetV2
    img_array = preprocess_input(img_array)

    # 5. Predict
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions[0])

    # 6. Map index to label
    labels_map = {v: k for k, v in class_indices.items()}
    return labels_map[predicted_class]


In [92]:
image_path = r'train\surprise\Training_320784.jpg'
prediction = predict_emotion(image_path, model, train_data.class_indices)
print("Predicted Emotion:", prediction)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
Predicted Emotion: surprise
